<a href="https://www.kaggle.com/visheshgoyal21/house-prices-advanced-regression-techniques?scriptVersionId=88614511" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
#Name: VISHESH GOYAL
#Roll No: 101903016
#Batch: 3COE1
#Email: vgoyal1_be19@thapar.edu

In [2]:
import cudf as cd
import cupy as np

In [3]:
dftest = cd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')
dftest.head()
dftrain = cd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
dftrain.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,<NA>,Reg,Lvl,AllPub,...,0,<NA>,<NA>,<NA>,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,<NA>,Reg,Lvl,AllPub,...,0,<NA>,<NA>,<NA>,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,<NA>,IR1,Lvl,AllPub,...,0,<NA>,<NA>,<NA>,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,<NA>,IR1,Lvl,AllPub,...,0,<NA>,<NA>,<NA>,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,<NA>,IR1,Lvl,AllPub,...,0,<NA>,<NA>,<NA>,0,12,2008,WD,Normal,250000


In [4]:
dftrain.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [5]:
dftrain.dtypes

Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
                  ...   
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
SalePrice          int64
Length: 81, dtype: object

In [6]:
dftrain.iloc[:,1:81].describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [7]:
dftrain.isnull().sum().sort_values(ascending=False)

PoolQC           1453
MiscFeature      1406
Alley            1369
Fence            1179
FireplaceQu       690
                 ... 
MoSold              0
YrSold              0
SaleType            0
SaleCondition       0
SalePrice           0
Length: 81, dtype: uint64

In [8]:
def cleaning_data_none(dftrain: dict, fields: dict):
    for field in fields:
        dftrain[field].fillna('None', inplace=True)

def cleaning_data_int(dftrain: dict, fields: dict):
    for field in fields:
        dftrain[field].fillna(0, inplace=True)

def cleaning_data_median(dftrain: dict, fields: dict):
    for field in fields:
        dftrain[field].fillna(dftrain[field].median(), inplace=True)

In [9]:
fields_clean_none = ['PoolQC',
                     'Alley',
                     'FireplaceQu',
                     'MasVnrType',
                     'Electrical',
                     'BsmtFinType2',
                     'BsmtFinType1',
                     'BsmtExposure',
                     'BsmtQual',
                     'BsmtCond',
                     'Fence',
                     'MiscFeature',
                     'GarageCond',
                     'GarageQual',
                     'GarageFinish',
                     'GarageType',
                     'SaleType',
                     'Utilities',
                     'Exterior1st',
                     'Exterior2nd',
                     'KitchenQual',
                     'Functional']

fields_clean_int = ['GarageYrBlt', 'MSZoning', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtFullBath', 'BsmtHalfBath']

fields_clean_median = ['LotFrontage',
                        'MasVnrArea',
                        'BsmtUnfSF',
                        'TotalBsmtSF',
                         'GarageCars',
                         'GarageArea']
                       

cleaning_data_none(dftrain, fields_clean_none)
cleaning_data_int(dftrain, fields_clean_int)
cleaning_data_median(dftrain, fields_clean_median)

In [10]:
features=dftrain.columns
features=list(features[1:len(features)-1])

In [11]:
len(features)

79

In [12]:
features

['MSSubClass',
 'MSZoning',
 'LotFrontage',
 'LotArea',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'MasVnrArea',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinSF1',
 'BsmtFinType2',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'KitchenQual',
 'TotRmsAbvGrd',
 'Functional',
 'Fireplaces',
 'FireplaceQu',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageCars',
 'GarageArea',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'WoodDeckSF',
 'OpenPorchSF',
 'Enc

In [13]:
dftrain.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000


In [14]:
import pandas as pd

In [15]:
df_types = pd.DataFrame(dftrain.dtypes, columns=["types"])
df_types_object = df_types[df_types["types"] == "object"]
for field_obj in df_types_object.index:
    dftrain[field_obj] = dftrain[field_obj].astype('category').cat.codes
dftrain.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,3,65.0,8450,1,1,3,3,0,...,0,3,4,1,0,2,2008,8,4,208500
1,2,20,3,80.0,9600,1,1,3,3,0,...,0,3,4,1,0,5,2007,8,4,181500
2,3,60,3,68.0,11250,1,1,0,3,0,...,0,3,4,1,0,9,2008,8,4,223500
3,4,70,3,60.0,9550,1,1,0,3,0,...,0,3,4,1,0,2,2006,8,0,140000
4,5,60,3,84.0,14260,1,1,0,3,0,...,0,3,4,1,0,12,2008,8,4,250000


In [16]:
dftrain['SalePrice'] = dftrain['SalePrice'].astype('float64')

In [17]:
Y = dftrain['SalePrice']
Y = np.array(Y)
Y.reshape(-1,1)
Y

array([208500., 181500., 223500., ..., 266500., 142125., 147500.])

In [18]:
X = dftrain[features]
X

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,3,65.0,8450,1,1,3,3,0,4,...,0,0,3,4,1,0,2,2008,8,4
1,20,3,80.0,9600,1,1,3,3,0,2,...,0,0,3,4,1,0,5,2007,8,4
2,60,3,68.0,11250,1,1,0,3,0,4,...,0,0,3,4,1,0,9,2008,8,4
3,70,3,60.0,9550,1,1,0,3,0,0,...,0,0,3,4,1,0,2,2006,8,0
4,60,3,84.0,14260,1,1,0,3,0,2,...,0,0,3,4,1,0,12,2008,8,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,3,62.0,7917,1,1,3,3,0,4,...,0,0,3,4,1,0,8,2007,8,4
1456,20,3,85.0,13175,1,1,3,3,0,4,...,0,0,3,2,1,0,2,2010,8,4
1457,70,3,66.0,9042,1,1,3,3,0,4,...,0,0,3,0,3,2500,5,2010,8,4
1458,20,3,68.0,9717,1,1,3,3,0,4,...,0,0,3,4,1,0,4,2010,8,4


In [19]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
x_scaled = scaler.fit_transform(X.as_matrix())

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x_scaled, Y, test_size = 0.3, random_state = 42)

In [21]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(1022, 79)
(438, 79)
(1022,)
(438,)


In [22]:
import cuml
from cuml import LinearRegression

In [23]:
X_test

array([[-0.87256276, -0.04553194,  0.00618985, ..., -1.36765473,
         0.31386709,  0.2085023 ],
       [ 0.07337496, -0.04553194,  1.27775352, ...,  1.64520971,
         0.31386709,  0.2085023 ],
       [-0.63607833,  1.53724502, -0.62959198, ...,  1.64520971,
         0.31386709,  0.2085023 ],
       ...,
       [ 0.07337496, -3.21108585, -0.22087509, ...,  0.13877749,
         0.31386709,  0.2085023 ],
       [-0.16310947, -0.04553194, -0.13004911, ...,  0.8919936 ,
        -2.26416922,  0.2085023 ],
       [ 0.07337496, -0.04553194,  1.55023145, ...,  0.8919936 ,
        -0.97515106,  1.11719888]])

In [24]:
lr = LinearRegression(fit_intercept = True, normalize = False, algorithm = 'eig')
reg = lr.fit(X_train, Y_train)
print("Coefficients:")
print(reg.coef_)
print("Intercept:")
print(reg.intercept_)
print("Predictions: ")
pred = lr.predict(X_test)
print(pred)
print("MSE: ")
print(cuml.metrics.regression.mean_squared_error(Y_test,pred))
print("R2 Score:")
print(cuml.metrics.regression.r2_score(Y_test,pred))
print("MAE:")
print(cuml.metrics.regression.mean_absolute_error(Y_test,pred))

Coefficients:
[ -5855.12187355  -1059.36938469  -4909.11850242   3537.81585952
   1391.30605537   1581.99176137  -1415.49755924   2436.76969184
  -1514.83544762    227.35544699   3102.62487358   2659.97555517
   -623.30620327  -4494.64035488  -2978.70975533  -3483.28587144
  15485.8025317    5335.5780853    6289.03901559    149.60285215
   2174.56435805   3106.04054534  -4881.31997679   2660.8936167
   2114.60436671   4473.14618194  -6364.52517353    678.3082903
    518.45990961  -7300.42553887   2432.12641817  -4504.91438644
  -1569.64831591    815.6852843     989.15890823    691.15498516
  -1848.67285861   -759.82284299   -947.53360687   -635.26834551
    183.50470116   -384.30324356   7987.28429378   7664.1446225
  -1375.06763188  12115.66294023   4364.13082931   -339.38973296
   1237.79648599   -403.25635722  -1982.73540038  -3319.92159267
  -7718.55663206   7945.82490154   4137.07069513   4052.67726763
  -1210.58371728   -945.99943403  -4061.29630986     87.17281171
  11706.789649

In [25]:
algorithm = ['svd', 'eig', 'qr', 'svd-qr', 'svd-jacobi']

In [26]:
for i in algorithm:
    print("Algorithm:")
    print(i)
    lr = LinearRegression(fit_intercept = True, normalize = False, algorithm = i)
    reg = lr.fit(X_train,Y_train)
    preds = lr.predict(X_test)
    print("MSE:")
    print(cuml.metrics.regression.mean_squared_error(Y_test,preds))
    print("R2 Score:")
    print(cuml.metrics.regression.r2_score(Y_test,preds))
    print("MAE:")
    print(cuml.metrics.regression.mean_absolute_error(Y_test,preds))

Algorithm:
svd
MSE:
1084070967.9947193
R2 Score:
0.844646546014768
MAE:
21873.24614382635
Algorithm:
eig
MSE:
1084070967.994126
R2 Score:
0.844646546014853
MAE:
21873.246143826007
Algorithm:
qr
MSE:
1071237580.3186837
R2 Score:
0.846485642495224
MAE:
21725.578795391633
Algorithm:
svd-qr
MSE:
2.9743476065761283e+33
R2 Score:
-4.262407053378082e+23
MAE:
4.345221127079089e+16
Algorithm:
svd-jacobi
MSE:
2.974347606576188e+33
R2 Score:
-4.262407053378169e+23
MAE:
4.345221127079123e+16


In [27]:
import cuml
from cuml import Ridge

In [28]:
alpha = np.array([1e-5])
ridge_eig = Ridge(alpha = alpha, fit_intercept = True, normalize = False, solver= "eig")

In [29]:
result_ridge = ridge_eig.fit(X_train, Y_train)
print("Coefficients: ")
print(result_ridge.coef_)
print("Intercept: ")
print(result_ridge.intercept_)

Coefficients: 
[-7.81118601e+14  2.84081335e+15  1.94033116e+15 -3.53233871e+14
  3.21995896e+14  2.32769122e+14  8.12514717e+15 -1.15270325e+14
 -2.97177636e+14 -1.51627167e+15  1.28775130e+16 -6.00563296e+15
 -3.95389035e+15  3.71607075e+14 -3.43326122e+15  7.04412026e+15
  1.22740212e+16  6.44244645e+14  2.55168224e+16 -9.67705247e+15
 -6.85495984e+13 -4.89588486e+15  1.61516924e+16 -1.61704074e+15
  8.98929549e+15 -3.88313299e+15 -8.95762103e+15 -3.80428769e+14
 -5.04578074e+15 -4.78595105e+15 -5.45053322e+15  2.07807210e+16
  7.20480783e+15 -6.79185333e+15  2.33396947e+16  1.45851191e+16
 -1.61980597e+16 -1.80127322e+16  1.90808217e+15  1.29846526e+16
  7.08708990e+15 -1.11005687e+16  2.22689060e+15 -1.35137591e+16
 -2.62327180e+12 -9.58813784e+15 -1.14707160e+15 -4.60955993e+14
 -1.24034325e+16  1.67011698e+15 -6.20954004e+15  1.11701105e+15
  6.10279898e+15  2.33696301e+16  8.24934571e+15 -6.74705885e+15
  5.25588072e+15  7.17827446e+15 -1.22939429e+16 -1.16313365e+16
 -3.297825

In [30]:
preds = result_ridge.predict(X_test)

In [31]:
print("MSE:")
print(cuml.metrics.regression.mean_squared_error(Y_test,preds))
print("R2 Score:")
print(cuml.metrics.regression.r2_score(Y_test,preds))
print("MAE:")
print(cuml.metrics.regression.mean_absolute_error(Y_test,preds))

MSE:
2.974347464306215e+33
R2 Score:
-4.262406849497312e+23
MAE:
4.345221026579623e+16


In [32]:
alpha = np.array([1e-5])
ridge_svd = Ridge(alpha = alpha, fit_intercept = True, normalize = False, solver= "svd")

In [33]:
result_ridge = ridge_svd.fit(X_train, Y_train)
print("Coefficients: ")
print(result_ridge.coef_)
print("Intercept: ")
print(result_ridge.intercept_)

Coefficients: 
[-7.81118601e+14  2.84081335e+15  1.94033116e+15 -3.53233871e+14
  3.21995896e+14  2.32769122e+14  8.12514717e+15 -1.15270325e+14
 -2.97177636e+14 -1.51627167e+15  1.28775130e+16 -6.00563296e+15
 -3.95389035e+15  3.71607075e+14 -3.43326122e+15  7.04412026e+15
  1.22740212e+16  6.44244645e+14  2.55168224e+16 -9.67705247e+15
 -6.85495984e+13 -4.89588486e+15  1.61516924e+16 -1.61704074e+15
  8.98929549e+15 -3.88313299e+15 -8.95762103e+15 -3.80428769e+14
 -5.04578074e+15 -4.78595105e+15 -5.45053322e+15  2.07807210e+16
  7.20480783e+15 -6.79185602e+15  2.33396947e+16  1.45851182e+16
 -1.61980623e+16 -1.80127296e+16  1.90808217e+15  1.29846526e+16
  7.08708990e+15 -1.11005687e+16  2.22689106e+15 -1.35137586e+16
 -2.62321351e+12 -9.58813847e+15 -1.14707160e+15 -4.60955993e+14
 -1.24034325e+16  1.67011698e+15 -6.20954004e+15  1.11701105e+15
  6.10279898e+15  2.33696301e+16  8.24934571e+15 -6.74705885e+15
  5.25588072e+15  7.17827446e+15 -1.22939429e+16 -1.16313365e+16
 -3.297825

In [34]:
preds = result_ridge.predict(X_test)

In [35]:
print("MSE:")
print(cuml.metrics.regression.mean_squared_error(Y_test,preds))
print("R2 Score:")
print(cuml.metrics.regression.r2_score(Y_test,preds))
print("MAE:")
print(cuml.metrics.regression.mean_absolute_error(Y_test,preds))

MSE:
2.9743474643062165e+33
R2 Score:
-4.2624068494973145e+23
MAE:
4.345221026579624e+16


In [36]:
from cuml.linear_model import Lasso
ls = Lasso(alpha = 0.1)

In [37]:
result_lasso = ls.fit(X_train, Y_train)
print("Coefficients:")
print(result_lasso.coef_)
print("intercept:")
print(result_lasso.intercept_)

Coefficients:
[-8.18796310e+14  2.83987861e+15  1.96445845e+15 -3.33239706e+14
  3.34383446e+14  2.39757195e+14  8.11540179e+15 -1.11191883e+14
 -3.08724708e+14 -1.51258232e+15  1.28838689e+16 -6.00888416e+15
 -3.95305148e+15  3.76832431e+14 -3.38728493e+15  7.06232419e+15
  1.24079666e+16  6.22452490e+14  2.55125417e+16 -9.67055852e+15
 -6.97709838e+13 -4.91175582e+15  1.61459003e+16 -1.61821516e+15
  8.98850755e+15 -3.88537457e+15 -8.95050562e+15 -3.86848163e+14
 -5.06479608e+15 -4.74436318e+15 -5.45073349e+15  2.07976150e+16
  7.20593356e+15 -1.23832227e+16  2.33440792e+16  1.26092887e+16
 -2.16072970e+16 -1.27249820e+16  1.90390295e+15  1.29997211e+16
  7.06833106e+15 -1.11233393e+16 -2.80469468e+15 -1.92055249e+16
 -6.31842180e+14 -2.84337808e+15 -1.13348075e+15 -4.54967498e+14
 -1.24361848e+16  1.65248513e+15 -6.19919129e+15  1.12689515e+15
  6.12001318e+15  2.34143392e+16  8.23603731e+15 -6.75004037e+15
  5.25482939e+15  7.18486955e+15 -1.23404323e+16 -1.16267119e+16
 -3.2789563

In [38]:
preds = result_lasso.predict(X_test)

In [39]:
print("MSE:")
print(cuml.metrics.regression.mean_squared_error(Y_test,preds))
print("R2 Score:")
print(cuml.metrics.regression.r2_score(Y_test,preds))
print("MAE:")
print(cuml.metrics.regression.mean_absolute_error(Y_test,preds))

MSE:
2.980079557780717e+33
R2 Score:
-4.2706212611559635e+23
MAE:
4.3485541660319464e+16
